---
title: "Research Methods II"
subtitle: "Session 3: Measuring Inequality"
author: Fernando Rios-Avila
format: 
  revealjs: 
    slide-number: true
    width: 1600
    height: 900
    code-fold: true
    echo: true
    css: styles.css 
    highlight-style: github
execute: 
  freeze: true
---


# Measuring Inequality

## What is Inequality?

- **Economic inequality** refers to how economic variables are distributed among individuals in a group, among groups in a population, or among countries. 
- Inequality of What? 
    - inequality of opportunities, for example access to employment or education
    - inequality of outcomes, for example material dimensions of human well-being, such as the level of income, educational attainment, health status and so on.

- For now we will focus on **income** inequality.

## How do you analyze (measure) inequality?

- There are various approaches that have been used for the analysis of Inequality

  - Intuitive approach
    - Unaxiomatic approach used to describe inequality.
  - Normative approach-Social welfare
    - Uses explicit concepts of welfare functions to quantify inequality 
  - Information theory
    - Quantifies inequality treating it as a problem of comparing income distribution probabilities.
  - Axiomatic approach
    - Uses a series of axioms to create measures of inequality

## Preliminaries

- Regardless of the approach, there are some basic steps required to measure inequality
  - Define the population of interest
  - Define the measure of interest
  - Adjust for prices (if necessary)
  - Adjust for individual heterogeneity (needs) (if necessary)

## Mathematical Preliminaries

- Let $y_i$ be the income of individual $i$ in the population. Assume that $y_i>>0$.
- Assume that $y_i$ can be characterized by a probability distribution function $f(y)$.
  
$$\begin{aligned}
y_i &\sim f(y) \rightarrow \int_{-\infty}^{z} f(y) dy = F(z) \\
F(0)&=0 \ \& \ F(\infty)=1 \\
F(Q_y(p)) &=p \rightarrow Q_y(p) = F^{-1}(p) 
\end{aligned}
$$

The $p_{th}$ quantile of $y_i$ is the value $Q_y(p)$ such that $p$ percent of the population has income below $Q_y(p)$.

## Mathematical Preliminaries

Mean of Standard of Living:

$$\mu_y = E(y) = \int_{-\infty}^{\infty} y f(y) dy=\int_0^1Q(p)dp
$$

Finally, the inequality measure can be written as:

$$I(y)=I(\mu_y, f(.)) = I(\mu_y, F(.))
$$

## Visualization tools

- There are several tools that can be used to visualize income distribution:
  - Density Function/Histogram
  - Pen parade/Cumulative Distribution Function
  - Lorenz Curve

## 

:::{.panel-tabset}

## Density/Histogram

- Density functions and histograms are used to visualize the distribution of income in the population.
- They could be used to detect multimodality, skewness, etc
- And could be used to compare distributions across groups.

- `Stata` Commands

  `histogram varname [weight] [if]`

  `kdensity varname [weight] [if]`

## Plots


In [ ]:
*| echo: true
*| code-fold: true
set scheme white2
set linesize 255
color_style tableau
qui:frause oaxaca, clear
sum wt, meanonly
gen int wt2 = round(wt/r(min))
qui:two histogram lnwage [fw=wt2] ///
    || kdensity lnwage [w=wt2], ///
    ysize(5) xsize(9) xtitle("Log Wages") ///
    legend(order(1 "Histogram" 2 "Kernel Density") pos(6) col(2)) 

:::

## {.scrollable} 

:::{.panel-tabset}

## PenParade/CDF 

- A different approach to visualize the distribution.
- The Pen Parade plots the values of the variable of interest in ascending order.
  - y-axis: Q(p); x-axis: p
- The CDF plots the cumulative distribution of the variable of interest.
  - y-axis: p; x-axis: Q(p)
- They give you a sense of the distribution, and easy comparison across high and low values.

## plot


In [ ]:
*| echo: true
*| code-fold: true
*| layout-ncol: 2
qui:pctile qlnwage = lnwage [w=wt], nq(100)
qui:gen  qwage = exp(qlnwage)
qui:gen p = _n if _n<100
scatter qwage p, connect(l) name(m1, replace) ysize(5) xsize(8) title("Pen Parade")
scatter p qwage, connect(l) name(m2, replace) ysize(5) xsize(8) title("Cumulative Distribution Function")

:::

##

:::{.panel-tabset}

## Lorenz Curve

- Perhaps the most popular tool to visualize income distribution.
- This curve plots the cumulative share of income vs the cumulative share of population.
  - How much of total income is held by the bottom X% of the population ?

- "Easy" to read:
  - The closest it is to the 45 degree line, the more equal the distribution.

- Not easy to implement with negative and zero incomes.

- Comparison across groups may be ambiguous.

- its always increasing at an increasing rate respect to X%

## Code 

Assume data is sorted by income

:::: {.columns}

::: {.column width="47%"}

x-axis: Cum share of population

$$P_j = \frac{\sum_{i=1}^j w_i}{\sum_{i=1}^n w_i}$$

:::

::: {.column width="47%"}
y-axis: Cum  share of income

$$LC_j = \frac{\sum_{i=1}^j y_i w_i}{\sum_{i=1}^n y_iw_i}$$
:::

::::


In [ ]:
*| echo: true
*| code-fold: false
*| output: false
frause oaxaca, clear
gen wage = exp(lnwage)
sort wage wt  // sort by income and weight 
// Estimate Totals for non missing data
egen twage = sum(wage * wt) if wage!=.
egen tpop  = sum(wt) 		if wage!=.
// get cumulative shares
gen lc_i = sum( (wage*wt/twage) )*100 if wage!=.
gen p_i  = sum( (wt/tpop) )*100      if wage!=.

## Plot


In [ ]:
*| echo: true
*| code-fold: true
*| fig-cap: "Lorenz Curve"
two (line lc_i p_i) /// Lorenz Curve
    ( function x, range(0 100) ) , /// 45 degree line
    aspect(1) ysize(5) xsize(8) ///
    xtitle("Cumulative Share of Population") ///
    ytitle("Cumulative Share of Income") ///
    legend(off)

## Alternative:

```stata
ssc install glcurve // installs command for Generalized Lorenz Curve
glcurve wage [aw = wt], /// provides variable and weight
    lorenz // Request ploting the Lorenz Curve
```

:::

## Inequality Measures

There are several measures of inequality. The most popular are:

- Interquantile Range (IQR) (or normalizations)
  $$IQR(\#1,\#2) = Q(\#2) - Q(\#1)
  $$

- Interquantile Share Ratio (Palma ratio (10/40))
  $$ISR(\#1,\#2) = \frac{1-LC(\#2)}{LC(\#1)}
  $$  

- Coefficient of Variation (CV)
  $$CV = \frac{\sigma_y}{\mu_y}
  $$

## Inequality Measures

- Lorenz Curve:

$$LC(p) = \frac{\int_0^p Q_y(u)du}{\int_0^1 Q_y(u)du} 
= \frac{1}{\mu_y} \int_0^p Q_y(u)du
$$

- Properties 1: Lorenz Curve is a non-decreasing function of $p$.

$$\frac{\partial LC(p)}{\partial p} = \frac{Q_y(p)}{\mu_y} \geq 0$$

- Properties 2: Lorenz Curve is a concave function of $p$ (increases at a fasterate).
$$\frac{\partial^2 LC(p)}{\partial p^2} = \frac{1}{\mu_y f(y)} \geq 0$$

## Inequality Measures: Gini Coefficient

- The Gini coefficient is the most popular measure of inequality.
- It is defined as (2x) the area between the Lorenz Curve and the 45 degree line.

$$Gini(y) = 2 \int_0^1 (p-LC(p)) dp$$

- where $p-LC(p)$ is the "loss" of income the Bottom $p$ percent of the population experiences.
  
- It is bounded between 0 (perfect Equality) and 1 (complete Inequality).
  
- When Lorenz do not cross, Gini provides unambiguous ranking of inequality.

$$Gini(y) = \frac{2}{\mu_y} Cov(y_p,p)$$


## Implementation {.scrollable}

- `Stata` has plenty of commands that can be used to estimate Gini
  - `search gini` for few examples
- I suggest 3 commands:
  - `fastgini` (`ssc install fastgini`)
  - `ineqdeco` (`ssc install ineqdeco`)
  - `sgini` (`ssc install sgini`)
  - `rif` (`ssc install rif`)

## {.scrollable}

In [ ]:
*| echo: true
*| code-fold: false
capture:ssc install sgini
sgini wage 

In [ ]:
*| echo: true
*| code-fold: false
capture:ssc install ineqdeco
ineqdeco wage [pw=wt]

## Other Inequality Measures

- There are other approaches that can be used to measure inequality.

  - Normative approach-Social welfare: Uses explicit concepts of welfare functions to quantify inequality

$$I_A(y,\varepsilon) = 1 - \left( \frac{1}{N} \sum_{i=1}^N \left(\frac{y_i}{\mu_y}\right)^{1-\varepsilon} \right)^\frac{1}{1-\varepsilon}
$$  

where \varepsilon is a measure of inequality aversion.
  
  - Information theory: Quantifies inequality treating it as a problem of comparing income distribution probabilities. How far are we from Full Entropy

$$I_{GE}(Y,\alpha)=\frac{1}{\alpha(1-\alpha)}\left[\frac{1}{N} \sum \left(\frac{y_i}{\mu_y}\right)^\alpha -1\right] 
$$

  - Axiomatic approach
    - Uses a series of axioms to create measures of inequality

# How to Compare Inequality 
a note

## Significance test

- As discussed in Session 1, we can use a t-test to compare means.
  - This requires estimating the standard error of the mean, use `mean` command, or `regress`
- Similarly, it may be as important to test whether two distributions (or inequality measures) are different.
  - This requires estimating the standard error of the inequality measure.
  - This is not as straightforward as the mean.
- Easiest methods:
  - **Bootstrap**: requires *bootstrap weights* for survey data.
  - **Influence function**: requires deriving the influence function of the inequality measure.  

## Bootstrap {.scrollable}

- Bootstrap its a non-parametric method to estimate the standard error of a statistic. Its based on Resampling and re-estimating data.
  
`bootstrap gini=r(coeff): sgini wage`


In [ ]:
*| echo: false
bootstrap gini=r(coeff), nodots: sgini wage

- RIF (Recentered Influence Function) is a method that uses the moment conditions to estimate the standard error of a statistic.

`rifhdreg wage , rif(gini)`


In [ ]:
*| echo: false
rifhdreg wage, rif(gini) robust

- Better yet, because you can use regressions (RIF-regressions), you can use weights, and test for differences in inequality across groups.

`rifhdreg wage ibn.female [pw=wt], rif(gini) over(female) noconstant`


In [ ]:
*| echo: false
rifhdreg wage ibn.female [pw=wt], rif(gini) over(female) noconstant

`rifhdreg wage i.female [pw=wt], rif(gini) over(female)`


In [ ]:
*| echo: false
rifhdreg wage i.female [pw=wt], rif(gini) over(female)

# How to Decompose Inequality


## Introduction

- Some times, we may be interested in determining what factors, and what extent, explain inequality.
  - But what do we mean by "explain"?
- There are several approaches to decompose inequality.
  - by sources: Explain how inequality is related to source of income
  - by groups: How much of the inequality is explained by inequality within groups, and between groups.
- We could also consider how inequality gaps are related to characteristics or returns to such characteristics.

## Decompose by sources

- Some inequality indices are well suited to decompose inequality by sources.
  - Variance Decomposition (Shorrocks 1982)
  - Gini Decomposition by source (Lerman and Yitzhaki 1985)

- The idea is assess inequality (or concentration) of each source of income, and then combine them to obtain their contribution to overall inequality.

##
### Variance Decomposition

Setup:
$$\begin{aligned}
Y &= y_1 + y_2 + \cdots + y_n  \\
I_V(Y) &= V(Y) = Cov(Y,Y) \\
 &= Cov(Y,y_1) + Cov(Y,y_2) + \cdots \\
\end{aligned}
$$

Covariance, however could be rewritten as: $Cov(Y,y_k)=\rho_k \sigma_Y \sigma_{y_k}$, thus

$$\begin{aligned}
V(y) = \sigma^2_Y &= \rho_1 \sigma_Y \sigma_{y_1} + \rho_2 \sigma_Y \sigma_{y_2} + \cdots \\
\sigma_Y&=   \rho_1 \sigma_{y_1} + \rho_2 \sigma_{y_2} + \cdots  \\
\end{aligned}
$$

Finally, if we divide all by $\mu_y$ and multiply by $\mu_{yk}/\mu_{yk}$, we get:

$$\begin{aligned}
\frac{\sigma_Y}{\mu_y} = CV(y) &=   \rho_1 \frac{1}{\mu_{y}} \frac{\mu_{y1}}{\mu_{y1}}\sigma_{y_1} + \rho_2 \frac{1}{\mu_{y}} \frac{\mu_{y2}}{\mu_{y2}}  \sigma_{y_2} + \cdots  \\
&= \rho_1 sh_1 CV(y_1) + \rho_2 sh_2 CV(y_2) + \cdots  \\
\end{aligned}
$$

## Example `Stata` {.scrollable}

`ssc install ineqfac`


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
frause limew1972.dta, clear
capture: ssc install ineqfac
ineqfac basemi inchomewealth incnonhomewealth govtr_n pubcon tx_n valhp [aw=hhwgt]

##
### Gini Decomposition by source

- The Gini index could also be decomposed by source of income. 

$$\begin{aligned}
Gini(y) &= \frac{2}{\mu_y} Cov(y,F(y)) = \frac{2}{\mu_y} Cov\left(\sum y_k,F(y)\right) =\frac{2}{\mu_y} \sum Cov\left( y_k,F(y)\right)
\end{aligned}
$$ 

- So Gini is the sum of the covariance of each source of income with $F(y)$  

$$\begin{aligned}
Gini(y) &= \sum \frac{2}{\mu_y} \times Cov( y_k,F(y)) \times \frac{\mu_{yk}}{\mu_{yk}} \times \frac{Cov( y_k,F(y_k))}{Cov ( y_k,F(y_k))} \\
&=\sum \frac{Cov ( y_k,F(y) )}{Cov ( y_k,F(y_k) )} \times \frac{2Cov( y_k,F(y_k)}{\mu_{yk}} \times \frac{\mu_{yk}}{\mu_y}  \\
&= \sum R_k \times G_k \times sh_k
\end{aligned} 
$$ 

##
### Gini Decomposition by source

$$\begin{aligned}
Gini(y)&= \sum R_k \times G_k \times sh_k
\end{aligned} 
$$ 

- $R_k$ = Gini Correlation; $G_k$ = Gini of source $k$; $sh_k$ = share of source $k$ in total income.
- $R_k\time G_k$ is the Concentration Index of $Y_k$

## Example `Stata` {.scrollable}

You probably  already have it installed

`ssc install sgini`


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
 
sgini basemi inchomewealth incnonhomewealth govtr_n pubcon tx_n valhp [aw=hhwgt], source

## Decompose by groups

- There is a second type of decomposition one may be interested in.
  - How much of the inequality is explained by inequality within groups, and between groups.
- For example, consider two cases:
  - Two (eq size) groups that have access to the same level of income, but within each group, all resources are held by one individual.
  - Two (eq size) groups, one has 80% of the income, and the other 20%, but within groups income is equally distributed.
- It is possible to understand the source of inequality by decomposing it by groups.
  - Entropy Indices (and Atkinson) are well suited for this type of decomposition. (see help ineqdeco)
  - GINI is not as straight forward but possible.

##
### GINI Decomposition by groups

- Decomposition of the GINI coefficient by groups Milanovic and Yitzhaki (2002)

- The method: To decompose the GINI by groups, one can use the following:

$$\begin{aligned}
Gini(y) &= \sum_{k=1}^K s_k O_k Gini(y_k) + Gini_{bw}
\end{aligned}
$$

- where $s_i$ is the share of group $i$ in total income, $Gini(y_k)$ is the Gini of group $k$, $O_k$ is a measure of overlapping across groups, and $Gini_{bw}$ is the Gini between groups.

$$Gini_{bw} = \frac{2}{\mu_y} Cov(\mu_i,\bar F_i)$$

##
### Overlapping 

- Overlapping $O_k$ measures to what extend the distribution of income in group $k$ overlaps with the distribution of income in other groups.
- If there is no overlapping, then $O_k=p_i$ (the population share of group k, and incomes are fully stratified). 
  
- Otherwise, this adjustment factors ensures that the sum of the Gini of each group is equal to the Gini of the total population + Between Gini.

## Example `Stata` {.scrollable}

`ssc install anogi` (Tom Masterson is one of the authors)  
`ssc install moremata` (needed for `anogi`)


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
capture:ssc install anogi
capture:ssc install moremata
anogi limew [aw= hhwgt ], by(educl) detail

# Thats all for today...
Until next week!